In [1]:
from lark import Lark

with open('grammar.lark') as f:
    grammar = f.read()
    
parser = Lark(grammar, start='input_file')
print(parser.parse(r'''
    MODULE main
    VAR
        x: {a, b, c};
    ASSIGN
        x := a & b = c;     
''').pretty())

input_file
  module_body
    var_list
      x
      type_specifier
        enum
          a
          b
          c
  module_body
    assign_list
      assign
        invar
          x
          expr
            basic_expr
              basic_expr	a
              &
              basic_expr
                basic_expr	b
                =
                basic_expr	c



In [2]:
with open('examples/1.smv') as f:
    tree = parser.parse(f.read())

In [3]:
from lark import Visitor, Token, Transformer

class VarDeclarations(Visitor):
    def __init__(self):
        self.boolean_variables = set()
        self.enum_variables = set()
        self.enum_values = set()
    
    def var_list(self, tree):
        if tree.children[1].children == []:
            self.boolean_variables.add(tree.children[0].value)
        else:
            self.enum_variables.add(tree.children[0].value)
            self.enum_values |= {token.value for token in tree.children[1].children[0].children}
            

visitor = VarDeclarations()
visitor.visit(tree)
visitor.boolean_variables, visitor.enum_variables, visitor.enum_values

({'lock'}, {'proc'}, {'critical', 'entering', 'exiting', 'idle'})

In [29]:
from syntax import *

class Expression(Transformer):
    def __init__(self, boolean_variables, enum_variables, enum_values):
        self.boolean_variables = boolean_variables
        self.enum_variables = enum_variables
        self.enum_values = enum_values
    
    def IDENTIFIER(self, data):
        if data in self.boolean_variables:
            return BooleanIdentifier(data)
        elif data in ['TRUE', 'FALSE']:
            return BooleanValue(data)
        elif data in self.enum_variables:
            return EnumIdentifier(data)
        elif data in self.enum_values:
            return EnumValue(data)


    def enum(self, children):
        return Enum(children)


    def basic_expr(self, children):
        match len(children):
            case 1:
                return children[0]
            case 2:
                return Not(children[1])
            case 3:
                match children[1].type:
                    case 'AND':
                        return And(children[0], children[2])
                    case 'OR':
                        return Or(children[0], children[2])
                    case 'IMPLIES':
                        return Implies(children[0], children[2])
                    case 'EQ':
                        return Eq(children[0], children[2])
                    case 'NEQ':
                        return Neq(children[0], children[2])
                    case 'IN':
                        return In(children[0], children[2])


    def case_body(self, children):
        if len(children) == 3:
            print(children[1])
            return ([children[0]] + children[2][0], [children[1]] + children[2][1], children[2][2])
        else:
            return ([children[0]], [children[1]], \
                "Boolean" if isinstance(children[1], BooleanExpression) else "Enum")


    def case_expr(self, children):
        match children[0][2]:
            case "Boolean":
                return BooleanCase(children[0][0], children[0][1])
            case "Enum":
                print(children[0][0], children[0][1])
                return EnumCase(children[0][0], children[0][1])


    def ltl(self, children):
        match len(children):
            case 1:
                return children[0]
            case 2:
                match children[0].type:
                    case 'G':
                        return G(children[1])
                    case 'F':
                        return F(children[1])
                    case 'X':
                        return X(children[1])
                    case 'Not':
                        return Not(children[1])
            case 3:
                match children[1].type:
                    case 'AND':
                        return And(children[0], children[2])
                    case 'OR':
                        return Or(children[0], children[2])
                    case 'IMPLIES':
                        return Implies(children[0], children[2])
                    case 'EQ':
                        return Eq(children[0], children[2])
                    case 'NEQ':
                        return Neq(children[0], children[2])
                    case 'IN':
                        return In(children[0], children[2])
            


expr = Expression(visitor.boolean_variables, visitor.enum_variables, visitor.enum_values)
expr.transform(tree)

[<syntax.And object at 0x7fe000e92ae0>, <syntax.Eq object at 0x7fe000e90470>, <syntax.Eq object at 0x7fe000e91610>, <syntax.Eq object at 0x7fe00199e0c0>, <syntax.BooleanValue object at 0x7fe00199edb0>] [<syntax.EnumValue object at 0x7fe000e901a0>, <syntax.EnumValue object at 0x7fe000e91430>, <syntax.Enum object at 0x7fe000e93770>, <syntax.EnumValue object at 0x7fe00199e900>, <syntax.EnumIdentifier object at 0x7fe00199ec30>]


Tree(Token('RULE', 'input_file'), [Tree(Token('RULE', 'module_body'), [Tree(Token('RULE', 'var_list'), [<syntax.BooleanIdentifier object at 0x7fe0185b2a50>, Tree(Token('RULE', 'type_specifier'), []), Tree(Token('RULE', 'var_list'), [<syntax.EnumIdentifier object at 0x7fe000e914c0>, Tree(Token('RULE', 'type_specifier'), [<syntax.Enum object at 0x7fe000e90d70>])])])]), Tree(Token('RULE', 'module_body'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'init'), [<syntax.EnumIdentifier object at 0x7fe000e90b90>, Tree(Token('RULE', 'expr'), [<syntax.EnumValue object at 0x7fe000e91b50>])])])]), Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'next'), [<syntax.EnumIdentifier object at 0x7fe000e90f80>, Tree(Token('RULE', 'expr'), [<syntax.EnumCase object at 0x7fe000e918e0>])])])]), Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'next'), [<syntax.BooleanIdentifier object at 0x7fe00

In [ ]:
from lark.visitors import Interpreter, Visitor

class MyInterpreter(Visitor):
    def __init__(self) -> None:
        super().__init__()
        self.invar = {}
    
    def assign(self, tree):
        print(tree)
        match tree.children[0].data:
            case 'init':
                print('init')
            case 'next':
                print('next')
            case 'invar':
                print('invar')
        
inter = MyInterpreter()

inter.visit(tree)

Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'init'), [Token('IDENTIFIER', 'proc'), Tree(Token('RULE', 'expr'), [Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'idle')])])])])
init
Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'next'), [Token('IDENTIFIER', 'proc'), Tree(Token('RULE', 'expr'), [Tree(Token('RULE', 'case_expr'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'basic_expr'), [Tree(Token('RULE', 'basic_expr'), [Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'proc')]), Token('EQ', '='), Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'idle')])]), Token('AND', '&'), Tree(Token('RULE', 'basic_expr'), [Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'lock')]), Token('EQ', '='), Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'FALSE')])])]), Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER'

Tree(Token('RULE', 'input_file'), [Tree(Token('RULE', 'module_body'), [Tree(Token('RULE', 'var_list'), [Token('IDENTIFIER', 'lock'), Tree(Token('RULE', 'type_specifier'), []), Tree(Token('RULE', 'var_list'), [Token('IDENTIFIER', 'proc'), Tree(Token('RULE', 'type_specifier'), [Tree(Token('RULE', 'enum'), [Token('IDENTIFIER', 'idle'), Token('IDENTIFIER', 'entering'), Token('IDENTIFIER', 'critical'), Token('IDENTIFIER', 'exiting')])])])])]), Tree(Token('RULE', 'module_body'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign_list'), [Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'init'), [Token('IDENTIFIER', 'proc'), Tree(Token('RULE', 'expr'), [Tree(Token('RULE', 'basic_expr'), [Token('IDENTIFIER', 'idle')])])])])]), Tree(Token('RULE', 'assign'), [Tree(Token('RULE', 'next'), [Token('IDENTIFIER', 'proc'), Tree(Token('RULE', 'expr'), [Tree(Token('RULE', 'case_expr'), [Tree(Token('RULE', 'case_body'), [Tree(Token('RULE', 'case_body'), [T

The recommended SAT-interface for this exercises is [pysat](https://pysathq.github.io/). If you installed the requirements, you should already have it.

In [ ]:
import pysat

from pysat.solvers import Cadical195 as Solver

g = Solver()
g.add_clause([-1, 2])
g.add_clause([-2, 3])
print(g.solve())
print(g.get_model())

True
[-1, -2, -3]
